In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy Agent Engine Agent on Agentspace

### Retrieve agents deployed on Agent Engine

The first step is to confirm that the agents that you want to deploy on Agentspace are functional in Agent Engine

[Docs](https://drive.google.com/file/d/1BSbt_UUNdzy5YvEVuDez7RihMdJ6pbul/view?resourcekey=0-IYbPfqERhbtRzSu_gvHy6A)

In [1]:
# Basic Libraries
import vertexai

# AI Engine on Vertex AI 
from vertexai import agent_engines

# To view JSON response formatted 
import json
from IPython.display import display,Markdown,JSON

In [3]:
# Authentication on gcloud
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=G6z1fHp7PSeApwCaWF4OkVTuVVNRyA&access_type=offline&code_challenge=w9TfpcnmLansTH7d21d1QhVyvFmyuOMWlOyFOSc4Bh0&code_challenge_method=S256


Credentials saved to file: [/Users/speca/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "speca-sandbox-argolis" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [4]:
# For Vertex AI SDK 
PROJECT_ID = "speca-sandbox-argolis"
LOCATION = "us-central1"
BUCKET = "gs://speca-argolis-adk-agents"

In [5]:
# Instantiate Vertex AI
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET,
)

In [6]:
## Retrieve all existent Agent Engine resource.names (Agents)
for agent in agent_engines.list():
    print(f"============================ \nAgent: {agent.display_name}\nResoruce Name: {agent.resource_name}\nCreated/updated at: {agent.update_time} \n\n" )

Agent: Fiscal Agent Gobierno de El Salvador
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/3333341023416877056
Created/updated at: 2025-06-16 03:34:31.816880+00:00 


Agent: Procurement Analytics Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/2791853536969752576
Created/updated at: 2025-06-05 17:22:44.496217+00:00 


Agent: Legal Analytics Agent
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/5506961162320740352
Created/updated at: 2025-05-31 15:40:31.286244+00:00 


Agent: Data Assistant
Resoruce Name: projects/809256159830/locations/us-central1/reasoningEngines/7923142362405011456
Created/updated at: 2025-05-31 12:08:18.375253+00:00 




In [7]:
# Retrieve the agent for testing
remote_agent = agent_engines.get("projects/809256159830/locations/us-central1/reasoningEngines/3333341023416877056")

In [8]:
# Run a simple query
for remote_event in remote_agent.stream_query(
    user_id="user_1",
    message="Hola",
):  pass
    #display(JSON(remote_event,expanded=True))

# Formatted output
display(Markdown(f"```json\n{json.dumps(remote_event, indent=2,ensure_ascii=False)}\n```"))

```json
{
  "content": {
    "parts": [
      {
        "text": "Hola, ¿en qué puedo ayudarte hoy con respecto a los datos de ejecución presupuestaria del Gobierno de El Salvador?"
      }
    ],
    "role": "model"
  },
  "usage_metadata": {
    "candidates_token_count": 25,
    "candidates_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 25
      }
    ],
    "prompt_token_count": 1486,
    "prompt_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 1486
      }
    ],
    "thoughts_token_count": 39,
    "total_token_count": 1550,
    "traffic_type": "ON_DEMAND"
  },
  "invocation_id": "e-8b9282e5-c162-4b2f-9df4-923c51293d07",
  "author": "goes_fiscal_agent",
  "actions": {
    "state_delta": {},
    "artifact_delta": {},
    "requested_auth_configs": {}
  },
  "id": "tpBP5jiT",
  "timestamp": 1750045266.142373
}
```

### Deploy Agent on Agentspace

After confirming that Agent is running OK on Agent Engine, there are two ways possible to deploy the agent running on Agent Engine on Agentspace: 

1) Deploy as an Agent on "My Agents" (from the doc: ) 
2) Deploy as a Company Agent (former option - still working)

Both methods is through API to publish some configuration on Agentspace.

Obs.: Some internal docs are doing this using CURL command, This notebook are using requests but is the same. 


In [9]:
# Libraries
import subprocess
import json
import requests

In [10]:
! gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=r0HvbVw73P1vXaxRpM4weqO8vWFUUy&access_type=offline&code_challenge=c6vQPPePSy531KyIKrAVcQPuh01ycEPSCh6fgRaEGiY&code_challenge_method=S256


You are now logged in as [admin@speca.altostrat.com].
Your current project is [speca-sandbox-argolis].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


### Basic Agent configuration

This step is needed for both options




In [11]:
# The ID of your Google Cloud project.
project_id = PROJECT_ID

# The ID of the Agentspace app (see: https://cloud.google.com/agentspace/agentspace-enterprise/docs/assistant ).
app_id = "agentspace-goes_1750043440771"

# The display name of the agent.
display_name = "Agente Transparencia Fiscal"

# The description of the agent, displayed on the frontend; it is only for the user’s benefit.
description = "Agente de datos abiertos del portal de transparencia fiscal"

# The description of the agent used by the LLM to route requests to the agent.
# Must properly describe what the agent does. Never shown to the user.
tool_description = "Useful agent to support El Salvador government fical analysts to identify insights and trends in data."

# The ID of the reasoning engine endpoint where the ADK agent is deployed (Resource.name).
adk_deployment_name = remote_agent.resource_name # The remote_agent.resource_name Deployed
 

###  (Option 1) - Deploy on "My Agents" list

This option will deploy the Agent on the users list (see the url: "...default_assistant/agents")

##### Configure the link between AS and AE Agent

In [12]:
# # Get the access token from gcloud
try:
    access_token = subprocess.check_output(
        "gcloud auth print-access-token", shell=True, text=True
    ).strip()
except subprocess.CalledProcessError as e:
    print(f"Error getting access token: {e}")
    exit()

# API endpoint
url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{app_id}/assistants/default_assistant/agents"

# Headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": project_id,
}

# Data payload
data = {
    "displayName": display_name,
    "description": description,
    "adk_agent_definition": {
        "tool_settings": {"tool_description": tool_description},
        "provisioned_reasoning_engine": {"reasoning_engine": adk_deployment_name},
    },
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print the response
print(f"Status Code: {response.status_code}")
print(f"Response JSON: \n")
JSON(response.json())

Status Code: 200
Response JSON: 



<IPython.core.display.JSON object>

##### Visualize configs

In [ ]:
# Get the access token from gcloud
try:
    access_token = subprocess.check_output(
        "gcloud auth print-access-token", shell=True, text=True
    ).strip()
except subprocess.CalledProcessError as e:
    print(f"Error getting access token: {e}")
    exit()

# API endpoint
url_get = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{app_id}/assistants/default_assistant/agents"

# Headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": project_id,
}

# Make the POST request
response = requests.get(url_get, headers=headers)

# Print the response
print(f"Status Code: {response.status_code}")
print(f"Response JSON: \n")
#display(JSON(response.json(),expanded=True))

# Envolvendo em um bloco de código Markdown para exibição
display(Markdown(f"```json\n{json.dumps(response.json(), indent=2)}\n```"))

#### Delete Agents

In [ ]:
agent_delete = "4199639121138852578"
# Get the access token from gcloud
try:
    access_token = subprocess.check_output(
        "gcloud auth print-access-token", shell=True, text=True
    ).strip()
except subprocess.CalledProcessError as e:
    print(f"Error getting access token: {e}")
    exit()

# API endpoint
url_get = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{app_id}/assistants/default_assistant/agents/{agent_delete}"

# Headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": project_id,
}

# Make the POST request
response = requests.delete(url_get, headers=headers)

# Print the response
print(f"Status Code: {response.status_code}")
print(f"Response JSON: \n")
#display(JSON(response.json(),expanded=True))

# Envolvendo em um bloco de código Markdown para exibição
display(Markdown(f"```json\n{json.dumps(response.json(), indent=2)}\n```"))

### (Option 2) Deploy on "Company Agents" list (DEPRECATED)

This option will deploy the Agent on the company list (see the url: "...default_assistant?updateMask=agentConfigs")

##### Configure the link between AS and AE Agent

In [ ]:
# # Get the access token from gcloud
# try:
#     access_token = subprocess.check_output(
#         "gcloud auth print-access-token", shell=True, text=True
#     ).strip()
# except subprocess.CalledProcessError as e:
#     print(f"Error getting access token: {e}")
#     exit()

# # API endpoint
# url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{app_id}/assistants/default_assistant?updateMask=agentConfigs"

# # Headers
# headers = {
#     "Authorization": f"Bearer {access_token}",
#     "Content-Type": "application/json",
#     "X-Goog-User-Project": project_id,
# }

# # Data payload
# data = {
#     "agentConfigs": [{
#         "displayName": display_name,
#         "toolDescription": tool_description,
#         "vertexAiSdkAgentConnectionInfo": {
#             "reasoningEngine": adk_deployment_name
#          },
     
#      "icon": {
#        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/search_insights/default/24px.svg"
#      },
#      "id": "legal_data_assistant"
#    }]
# }

# # Make the POST request
# response = requests.patch(url, headers=headers, data=json.dumps(data))

# # Print the response
# print(f"Status Code: {response.status_code}")
# print(f"Response JSON: \n")
# #display(JSON(response.json(),expanded=True))

# # Envolvendo em um bloco de código Markdown para exibição
# display(Markdown(f"```json\n{json.dumps(response.json(), indent=2)}\n```"))

##### Visualize configs

In [ ]:
# Get the access token from gcloud
try:
    access_token = subprocess.check_output(
        "gcloud auth print-access-token", shell=True, text=True
    ).strip()
except subprocess.CalledProcessError as e:
    print(f"Error getting access token: {e}")
    exit()

# API endpoint
url_get = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{app_id}/assistants/default_assistant"

# Headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": project_id,
}

# Make the POST request
response = requests.get(url_get, headers=headers)

# Print the response
print(f"Status Code: {response.status_code}")
print(f"Response JSON: \n")
#display(JSON(response.json(),expanded=True))

# Envolvendo em um bloco de código Markdown para exibição
display(Markdown(f"```json\n{json.dumps(response.json(), indent=2)}\n```"))

#### Remove Agent

In [ ]:
# Get the access token from gcloud
try:
    access_token = subprocess.check_output(
        "gcloud auth print-access-token", shell=True, text=True
    ).strip()
except subprocess.CalledProcessError as e:
    print(f"Error getting access token: {e}")
    exit()

# API endpoint
url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{app_id}/assistants/default_assistant?updateMask=agentConfigs"

# Headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": project_id,
}

# Data payload
data = {
#     "agentConfigs": [{
#         "displayName": display_name,
#         "toolDescription": tool_description,
#         "vertexAiSdkAgentConnectionInfo": {
#             "reasoningEngine": adk_deployment_name
#          },
     
#      "icon": {
#        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/search_insights/default/24px.svg"
#      },
#      "id": "legal_data_assistant"
#    }]
}

# Make the POST request
response = requests.patch(url, headers=headers, data=json.dumps(data))

# Print the response
print(f"Status Code: {response.status_code}")
print(f"Response JSON: \n")
#display(JSON(response.json(),expanded=True))

# Envolvendo em um bloco de código Markdown para exibição
display(Markdown(f"```json\n{json.dumps(response.json(), indent=2)}\n```"))